In [26]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.io import preprocess_data
import matplotlib.gridspec as gridspec
import numpy as np
from paths import TRAINING_DATA_PATH,EXAMPLE_PATH
import glob
import simpa as sp
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import umap
from matplotlib.colors import LinearSegmentedColormap
import patato as pat
import nrrd

In [2]:
PATH = r"I:\research\seblab\data\group_folders\Janek\Mouse_TerminalCO2\processed/"

In [3]:
scans = glob.glob(PATH + "Scan_*.hdf5")

In [7]:
timesteps = [0, 5, 10, -15, -10, -5]

In [43]:
for scan in scans:
    scan_name = scan.split("\\")[-1].split(".")[0]
    print(scan_name)
    pa_data = pat.PAData.from_hdf5(scan)    
    recon = pa_data.get_scan_reconstructions()[('OpenCL Backprojection', '0')][timesteps, :, :, :, :]
    
#     np.savez(PATH + "/" + scan_name + "_slices.npz", recon=recon.raw_data)
#     nrrd.write(PATH + "/" + scan_name + "_slices.nrrd", recon.raw_data[:, 4, :, :])

    unmixer = pat.SpectralUnmixer(chromophores=["Hb", "HbO2"],
                                  wavelengths = pa_data.get_wavelengths(), rescaling_factor=1)
    so2_calculator = pat.SO2Calculator(nan_invalid=True)

    unmixed, _, _ = unmixer.run(recon, None)
    so2, _, _ = so2_calculator.run(unmixed, None)
    
    nrrd.write(PATH + "/" + scan_name + "_sO2.nrrd", so2.raw_data[:, 0, :, :])

Scan_1
Scan_2
Scan_3
Scan_5
Scan_6
Scan_7


In [45]:
wl = pa_data.get_wavelengths()

In [50]:
for scan in scans:
    scan_name = scan.split("\\")[-1].split(".")[0]
    label_path = scan.replace(".hdf5", "_slices-labels.nrrd")
    spectra_path = scan.replace(".hdf5", "_slices.npz")
    lu_path = scan.replace(".hdf5", "_sO2.nrrd")
    labels, _ = nrrd.read(label_path)
    lu, _ = nrrd.read(lu_path)
    spectra = np.load(spectra_path)["recon"]
    
    np.savez(scan.replace("Scan", "CO2").replace(".hdf5", ".npz"),
            lu=lu,
            reference_mask=labels,
            spectra=spectra,
            wavelengths=wl)